In [1]:
import os
HOME = os.getcwd()
print(HOME)

/kaggle/working


In [2]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.20 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5519.7/8062.4 GB disk)


In [3]:
from ultralytics import YOLO
from IPython.display import display, Image

# Import datasets from roboflow

In [5]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

/kaggle/working/datasets


In [ ]:
%cd {HOME}/datasets
!pip install roboflow
from roboflow import Roboflow

rf = Roboflow(api_key="A88aglIXim3OCSF5vNOe")
project = rf.workspace("gabriel-advent-hobto").project("detect-tdz36")
version = project.version(1)
dataset = version.download("yolov8")


# Train Grid Search

In [5]:
import subprocess
from itertools import product

param_grid = {
    'model': ['yolov8s.pt'],
    'dropout': [0.2, 0.5],
    'batch': [32, 64],
    'lr': [0.001, 0.0001],
    'optimizer' : ['Adam', 'RMSProp'],
}

# Generate semua kombinasi
param_combinations = list(product(*param_grid.values()))
print(len(param_combinations))

32


In [ ]:
# Memasukan semua kombinasi parameter ke dalam commands

commands = []
for params in param_combinations:
    model, dropout, batch, lr, optimizer = params
    command = f"yolo detect train \
data='/kaggle/working/datasets/detect-2/data.yaml' \
model='{model}' \
epochs=200 \
imgsz='416' \
optimizer={optimizer} \
lr0={lr} \
batch='{batch}' \
dropout='{dropout}'\
plots=True \
save=True"
    commands.append(command)

In [ ]:
%cd {HOME}

import subprocess

# Loop setiap perintah dan jalankan menggunakan subprocess
for command in commands:
    print(command)
    subprocess.run(command, shell=True)

# Validasi

In [ ]:
%cd {HOME}

!yolo task=detect mode=val model='/kaggle/working/runs/detect/train/weights/best.pt' data='/kaggle/working/datasets/detect-2/data.yaml' plots=True save_json=True

# Testing

In [ ]:
import glob
import pandas as pd
from PIL import Image as PILImage
from IPython.display import display
import cv2
import numpy as np

In [ ]:
# Path to the test images
test_path = "/kaggle/working/detect-1/test/images"

# Load the YOLO model
model = YOLO(f"/kaggle/working/runs/detect/train/weights/best.pt")

# Perform predictions
results = model.predict(source=test_path)
names = model.names
labels = []

# DataFrame to store detection results
df_results = pd.DataFrame(columns=['Image', 'Labels'])

for r in results:
    # Get bounding boxes and class indices
    boxes = r.boxes.xyxy
    cls = r.boxes.cls
    
    # Sort bounding boxes and class indices by x1 coordinate
    sorted_indices = boxes[:, 0].argsort()
    sorted_boxes = boxes[sorted_indices]
    sorted_cls = cls[sorted_indices]
    
    # Get class names
    file_labels = [names[int(c)] for c in sorted_cls]
    
    # Append labels to list
    labels.append((file_labels, sorted_boxes, sorted_cls))

In [ ]:
i = 0

# List of supported image extensions
image_extensions = ['png', 'jpg', 'jpeg']

for image_extension in image_extensions:
    image_paths = glob.glob(f"{test_path}/*.{image_extension}")
    for image_path in sorted(image_paths):
        print(image_path)
        file_labels, sorted_boxes, sorted_cls = labels[i]
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Draw bounding boxes on the image
        for box, cls_idx in zip(sorted_boxes, sorted_cls):
            x1, y1, x2, y2 = map(int, box)
            label = names[int(cls_idx)]
            color = (0, 255, 0)  # Green color for bounding box
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
            cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
        
        # Display the image with bounding boxes
        display(PILImage.fromarray(img))
        
        # Print the detection labels
        print("Detected Labels:", ", ".join(file_labels))
        print("\n")
        
        # Add detection results to the DataFrame
        df_results = pd.concat([df_results, pd.DataFrame({'Image': [image_path], 'Labels': [', '.join(file_labels)]})], ignore_index=True)
        
        i += 1

# Save the DataFrame to an Excel file
df_results.to_excel("/kaggle/working/detection_results.xlsx", index=False)